In [1]:
import pandas as pd
import torchvision.transforms as tt
from torch.utils.data import Dataset
import os
import numpy as np
import torchvision
from PIL import Image

import torch
from torchvision import models
from torch.utils.data import DataLoader

In [2]:
class GDataset(Dataset):
    def __init__(self, df, directory):

        self.df = df
        self.dir = directory
        self.transform = tt.Compose([
            tt.Resize((160, 160)),
            tt.RandomCrop((128, 128)),
            tt.ToTensor(),
            tt.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
        ])
    
    
    def __len__(self):
        return len(self.df)

    
    def __getitem__(self, index):
        
        id_ = self.df.iloc[index]['id']
        img_name = "/".join([self.dir, id_[:3], id_+".jpg"])
        image = Image.open(img_name)
        image = self.transform(image)
        return image, torch.tensor(int(self.df.iloc[index]['class_id'])).long()
        

In [3]:
valid = pd.read_csv("./valid_2.csv")
valid_data = GDataset(valid, "train")

batch_size = 100
valid_loader = DataLoader(valid_data, batch_size=batch_size)

# model = models.vgg16(pretrained=True)
model = models.vgg16(pretrained=False)
model.avgpool = torch.nn.AdaptiveAvgPool2d(output_size=(3, 3))
model.classifier = torch.nn.Sequential(torch.nn.Linear(4608, 4096),
                                       torch.nn.ReLU(),
                                       torch.nn.Linear(4096, 4096),
                                       torch.nn.ReLU(),
                                       torch.nn.Linear(4096, valid['class_id'].max() + 1))
model.load_state_dict(torch.load("model27.pth"))
model.cuda()
criterion = torch.nn.CrossEntropyLoss()

softmax = torch.nn.Softmax(dim=1)

loss_sum = 0
accuracy = 0
top_5_accuracy = 0
counter = 0
for x, t in valid_loader:
    counter += 1
    print(round(counter / len(valid_loader), 2), end="\r")
    x, t = x.cuda(), t.cuda()
    z = model(x)
    loss = criterion(z, t)
    loss_sum += loss.item()

    y = softmax(z)
    top_p, top_class = y.topk(5, dim=1)
    accuracy += (top_class[:, 0] == t).sum().item()
    top_5_accuracy += (top_class == t.unsqueeze(1).repeat(1, 5)).max(axis=1).values.sum().item()

print(loss_sum / len(valid_loader), 
      accuracy / len(valid_data), 
      top_5_accuracy / len(valid_data))
   

5.868094732003565 0.25145163163395656 0.42901521309952384
